In [41]:
import pandas as pd
import numpy as np
import datetime
import sys, os
import glob
import time
import fnmatch
import time
import xlwings as xw
start_time = time.time()
#read inputs
#consensus tab
consensus1 = pd.read_excel("Pre consensus - example.xlsx", dtype={'Product Family':str,'Region':str},sheet_name = 'S&OP',
                           header=2, keep_default_na=False)

#fill rate tab
fillrate = pd.read_excel("Pre consensus - example.xlsx",sheet_name = 'FILL RATE', header=2, 
                         usecols = "A,C,U,V,X,AA,AB,AC,AD,AF,AH,AJ")
fillrate.rename(columns={ fillrate.columns[6]: "SFDC+1"}, inplace = True)

LRP = pd.read_excel("Pre consensus - example.xlsx",sheet_name = 'LRP', header=4, 
                         usecols = "A,D,E")
LRP.rename(columns={'PRODUCT FILTER':'Product Family'}, inplace=True)
LRP.rename(columns={ LRP.columns[1]: "LRP",LRP.columns[2]: "LRP_1" }, inplace = True)
#create world wide rows
#select the column
ww = consensus1[["Product Family"]] #df1 = df1[(df1['EVM_AIT2'] != 'EVM')]
#drop duplicates'
ww = ww.drop_duplicates()
ww['Region'] = 'WW' #add region as WW
#append ww row to original file S&PP
consensus= consensus1.append(ww,ignore_index = True)
#make product family as ww for formula in excel
 #consensus['Product Family'] =np.where(consensus['Region'] == 'WW','WW',consensus['Product Family'])

#add columns
consensus.insert(0, 'Demand Planner', '')
consensus.insert(1, 'FAM & REG', consensus['Product Family']+consensus['Region'])
consensus.insert(2, 'Business', 'EVM')
consensus.drop(labels=['Sum of SFDC'], axis=1, inplace = True) #for this column we'll use info from fill rate

fillrate.rename(columns={ fillrate.columns[7]: "Load.1"}, inplace = True)
fillrate=fillrate.drop_duplicates(subset='FAM & REG',keep='first')
preconsensus = pd.merge(consensus, fillrate, on='FAM & REG', how='left')

#reorder columns as needed
mid = preconsensus['Group']
preconsensus.drop(labels=['Group'], axis=1, inplace = True)
preconsensus.insert(3, 'Group', mid)
preconsensus.insert(4, 'Technology', '')
mid = preconsensus['ASP']
mid2 = preconsensus['Sum of 4 Qtr Shipment']
preconsensus.drop(labels=['ASP','Sum of 4 Qtr Shipment'], axis=1, inplace = True)
preconsensus.insert(17, 'SM-NM', '')

#new name for column 20 as its not possible to have 2 columns under same name 
#make sure Q2'21 Shipped from S&OP doesn have empty values
name17c=preconsensus.iloc[:, 12].name
name17c = name17c + "_2"
#name17c

preconsensus[preconsensus.columns[12]].fillna(0, inplace = True)
preconsensus[preconsensus.columns[12]] = preconsensus[preconsensus.columns[12]].astype(int)
preconsensus.insert(20, name17c, preconsensus[preconsensus.columns[12]])
preconsensus.insert(16, 'Sum of 4 Qtr Shipment', mid2)

#extract number3 to get the four
name26c=preconsensus.iloc[:, 26].name
name26col = name26c[8]
name26col =int(name26col)+1
name26col = str (name26col) + "'"
name26col = 'Q' + name26col + "2022 Total Demand"
preconsensus.insert(27, name26col, '')

preconsensus = pd.merge(preconsensus, LRP, on='Product Family', how='left')
mid = preconsensus['LRP']
preconsensus.drop(labels=['LRP'], axis=1, inplace = True)
preconsensus.insert(28, 'LRP', mid)
preconsensus.insert(29,'Disti DOH','')

#get previous PM Outlook from last week report
#make sure tehre are no grouped or hidden columns
consensuslastwk = pd.read_excel("Pre Consensus Template.xlsx",header=1,dtype={'FAM & REG':str}) #change file form last week
consensuslastwk = consensuslastwk[['FAM & REG', 'PM Outlook','PM Outlook.1','PM Outlook.2','PM Outlook.3']]
consensuslastwk.rename(columns={'PM Outlook':'Prev PM OUTLOOK','PM Outlook.1':'Prev PM OUTLOOK.1',
                                'PM Outlook.2':'Prev PM OUTLOOK.2','PM Outlook.3':'Prev PM OUTLOOK.3'}, inplace=True)
#print(consensuslastwk.shape, 'rows of prev preconsensus')
consensuslastwk=consensuslastwk.drop_duplicates(subset='FAM & REG',keep='first')
#print(consensuslastwk.shape, 'rows of prev preconsensus without duplicates')
preconsensus = pd.merge(preconsensus, consensuslastwk, on='FAM & REG', how='left')

mid = preconsensus['Prev PM OUTLOOK']
preconsensus.drop(labels=['Prev PM OUTLOOK'], axis=1, inplace = True)
preconsensus.insert(30, 'Prev PM OUTLOOK', mid)

preconsensus.insert(31,'PM Outlook','')
preconsensus.insert(32,'????',0)
preconsensus.insert(33,'M/M Cons Chg',1)
preconsensus['M/M Cons Chg'] = np.where( preconsensus['????']== 0,0,
                                        (preconsensus.iloc[: , 12]-preconsensus['????'])/preconsensus['????'])                                 

preconsensus.insert(35,'Fill%',np.where( preconsensus.iloc[: , 12] ==0,0,
                                        preconsensus['Sum of Curr Qtr Backlog']/preconsensus.iloc[: , 12]))
preconsensus.rename(columns={'Sum of Curr Qtr Backlog':'Load'}, inplace=True)

mid = preconsensus['Sum of SFDC']
preconsensus.drop(labels=['Sum of SFDC'], axis=1, inplace = True)
preconsensus.insert(36, 'CRM', mid)

# column reference must be shipped_2, new column created

print(preconsensus.iloc[: , 21].name,'reference must be shipped_2')
preconsensus.insert(37,'Fill(CRM)%',np.where( preconsensus.iloc[: , 21] == 0,0,
                                        (preconsensus['CRM']+preconsensus['Load'])/preconsensus.iloc[: , 21]))

#sfr input, curren Quater as Q, next quarter as Q1, until Q2
SFR = pd.read_excel('SFR.xlsx',sheet_name = 'report')
SFR.rename(columns={ 'Named Product FamilyTheater': "NamedProductFamilyTheater"}, inplace = True)
#sum, quantities
df14 = pd.crosstab(index=[SFR.NamedProductFamilyTheater], columns=[SFR["Sales Stage"],SFR.Quarter], values=SFR["Quantity"], aggfunc='sum').fillna(0)
df14.rename(columns = {"Q":"Q0"}, inplace = True)
#df14.head()
#df14.columns
##df14 = pd.crosstab(index=[SFR.NamedProductFamilyTheater,SFR.Quarter], columns=[SFR["Sales Stage"]], values=SFR["Quantity"], aggfunc='sum').fillna(0)

#SFR QTY
Q=df14.filter(like="Q0", axis=1)
#drop extra level from multi index
Q = Q.droplevel('Quarter', axis = 1)
#Q.columns
# delete extra classifications
for i in Q.columns:    
 if i == 'Closed - Won':
    Q.drop(labels = [i],axis = 1, inplace = True)
print(Q.columns)
#Q.head()

Q.reset_index(inplace=True) #CREATE NEW INDEXES
Q.rename(columns={'NamedProductFamilyTheater':'FAM & REG'}, inplace=True)
#Q.drop("Sales Stage",axis=1,inplace=True)
Q.head()

#SFR QTY Q1
Q1=df14.filter(like="Q+1", axis=1)
Q1 = Q1.droplevel('Quarter', axis = 1)
#rename columns for Q1
Q1.rename(columns={'Identify':'IdentifyQ1','Qualify':'QualifyQ1',
                                'Secure':'SecureQ1','Validate':'ValidateQ1'}, inplace=True)

#DELETE not needed columns in case they exist
for i in Q1.columns:
    
 if i == 'Closed - Won':
    #print(i)
    Q1.drop(labels = [i],axis = 1, inplace = True)
#Q1.columns
#Q1.head()
Q1.reset_index(inplace=True) #CREATE NEW INDEXES
Q1.rename(columns={'NamedProductFamilyTheater':'FAM & REG'}, inplace=True)
#Q.drop("Sales Stage",axis=1,inplace=True)
Q1.head()

#MERGE SFR SUM QTY Q AND Q1
QX = pd.merge(Q, Q1, on='FAM & REG', how='left')
QX.head()

#SFR QTY Q2
Q2=df14.filter(like="Q+2", axis=1)
Q2 = Q2.droplevel('Quarter', axis = 1)
#Q2.columns
#Q2.head()
Q2.rename(columns={'Identify':'IdentifyQ2','Qualify':'QualifyQ2',
                                'Secure':'SecureQ2','Validate':'ValidateQ2'}, inplace=True)
for i in Q2.columns:
    if i == 'Closed - Won':
     Q2.drop(labels = [i],axis = 1, inplace = True)
    
Q2.reset_index(inplace=True) #CREATE NEW INDEXES
Q2.rename(columns={'NamedProductFamilyTheater':'FAM & REG'}, inplace=True)
#Q.drop("Sales Stage",axis=1,inplace=True)
Q2.head()

#MERGE SFR SUM Q/Q1 AND Q2
QX = pd.merge(QX, Q2, on='FAM & REG', how='left')
#QX.head()

#count SFR
qcount = pd.crosstab(index=[SFR.NamedProductFamilyTheater], columns=[SFR["Sales Stage"],SFR.Quarter], values=SFR["Quantity"], aggfunc='count').fillna(0)
qcount.rename(columns = {"Q":"Q0"}, inplace = True)
#qcount.head()

#COUNT SFR Q
Qcount=qcount.filter(like="Q0", axis=1)
#drop extra level from multi index
Qcount = Qcount.droplevel('Quarter', axis = 1)
#Q.columns
# delete extra classifications
for i in Qcount.columns:    
 if i == 'Closed - Won':
    Qcount.drop(labels = [i],axis = 1, inplace = True)
print(Qcount.columns)

Qcount.reset_index(inplace=True) #CREATE NEW INDEXES
Qcount.rename(columns={'NamedProductFamilyTheater':'FAM & REG'}, inplace=True)
Qcount.rename(columns={'Identify':'Identifycount','Qualify':'Qualifycount',
                                'Secure':'Securecount','Validate':'Validatecount'}, inplace=True)
#Q.drop("Sales Stage",axis=1,inplace=True)
Qcount.head()

#COUNT SFR Q1
Qcount1=qcount.filter(like="Q+1", axis=1)
Qcount1 = Qcount1.droplevel('Quarter', axis = 1)
#rename columns for Q1
Qcount1.rename(columns={'Identify':'IdentifycountQ1','Qualify':'QualifycountQ1',
                                'Secure':'SecurecountQ1','Validate':'ValidatecountQ1'}, inplace=True)

#DELETE not needed columns in case they exist
for i in Qcount1.columns:
    
 if i == 'Closed - Won':
    #print(i)
    Qcount1.drop(labels = [i],axis = 1, inplace = True)
Qcount1.reset_index(inplace=True) #CREATE NEW INDEXES
Qcount1.rename(columns={'NamedProductFamilyTheater':'FAM & REG'}, inplace=True)
#Q.drop("Sales Stage",axis=1,inplace=True)
Qcount1.head()

#MERGE COUNT SFR Q AND Q1
QXcount = pd.merge(Qcount, Qcount1, on='FAM & REG', how='left')
QXcount.head()

#SFR COUNT Q2
Qcount2=qcount.filter(like="Q+2", axis=1)
Qcount2 = Qcount2.droplevel('Quarter', axis = 1)
Qcount2.rename(columns={'Identify':'IdentifycountQ2','Qualify':'QualifycountQ2',
                                'Secure':'SecurecountQ2','Validate':'ValidatecountQ2'}, inplace=True)
for i in Qcount2.columns:
    
 if i == 'Closed - Won':
    #print(i)
    Qcount2.drop(labels = [i],axis = 1, inplace = True)

Qcount2.reset_index(inplace=True) #CREATE NEW INDEXES
Qcount2.rename(columns={'NamedProductFamilyTheater':'FAM & REG'}, inplace=True)
#Q.drop("Sales Stage",axis=1,inplace=True)
Qcount2.head()

#MERFE SFR SUM Q/Q1 AND Q2
QXcount = pd.merge(QXcount, Qcount2, on='FAM & REG', how='left')
#QXcount.head()

SFRQTY_COUNT = pd.merge(QX, QXcount, on = 'FAM & REG', how='left')
SFRQTY_COUNT.head()

preconsensus.insert(38,'Secure Qty','')
preconsensus.insert(39,'Secure Count','')
preconsensus.insert(40,'Validate Qty','')
preconsensus.insert(41,'Validate Count','')
preconsensus.insert(42,'Qualify Qty','')
preconsensus.insert(43,'Qualify Count','')
preconsensus.insert(44,'Identify Qty','')
preconsensus.insert(45,'Identify Count','')
#merge sfr pivots
preconsensus = pd.merge(preconsensus, SFRQTY_COUNT, on = 'FAM & REG', how='left')
#arrange qty and count
preconsensus['Secure Qty']=preconsensus['Secure']
preconsensus = preconsensus.drop(['Secure'], axis = 1)

preconsensus['Secure Count']=preconsensus['Securecount']
preconsensus = preconsensus.drop(['Securecount'], axis = 1)

preconsensus['Validate Qty']=preconsensus['Validate']
preconsensus = preconsensus.drop(['Validate'], axis = 1)

preconsensus['Validate Count']=preconsensus['Validatecount']
preconsensus = preconsensus.drop(['Validatecount'], axis = 1)

preconsensus['Qualify Qty']=preconsensus['Qualify']
preconsensus = preconsensus.drop(['Qualify'], axis = 1)

preconsensus['Qualify Count']=preconsensus['Qualifycount']
preconsensus = preconsensus.drop(['Qualifycount'], axis = 1)

preconsensus['Identify Qty']=preconsensus['Identify']
preconsensus = preconsensus.drop(['Identify'], axis = 1)

preconsensus['Identify Count']=preconsensus['Identifycount']
preconsensus = preconsensus.drop(['Identifycount'], axis = 1)
#end columns SFR CURRENT Q
preconsensus.insert(46,'Wgt Funnel + Load',0)
preconsensus.insert(47,'Totl Funnel + Load',0)
preconsensus.insert(48,'Total SFDC Count',0)
preconsensus.insert(49,'Wgt Funnel As % of Tot Dmd',np.where( preconsensus.iloc[: , 12] ==0,0,
                                        preconsensus['Wgt Funnel + Load']/preconsensus.iloc[: , 12]))
preconsensus.insert(50,'Tot Funnel As % of Tot Dmd',0)
preconsensus.insert(51,'Wgt Funnel+Load Fill',0)
preconsensus.insert(52,'Demand var',0)

#Q+1
mid = preconsensus['LRP_1']
preconsensus.drop(labels=['LRP_1'], axis=1, inplace = True)
preconsensus.insert(53, 'LRP.1', mid)
#Prev PM Outlook
mid = preconsensus['Prev PM OUTLOOK.1']
preconsensus.drop(labels=['Prev PM OUTLOOK.1'], axis=1, inplace = True)
preconsensus.insert(54, 'Prev PM OUTLOOK.1', mid)
preconsensus.insert(55,'PM Outlook.1',0)
preconsensus.insert(56,'M/M Cons Chg.1',0)
mid = preconsensus['Load.1']
preconsensus.drop(labels=['Load.1'], axis=1, inplace = True)
preconsensus.insert(57, 'Load.1', mid)

preconsensus['Load.1'].fillna(0, inplace = True)
preconsensus['Load.1']= preconsensus['Load.1'].astype(int)
#preconsensus['Load.1'].dtype
preconsensus['Sum of Curr Qtr Total Demand'].fillna(0, inplace = True)
preconsensus['Sum of Curr Qtr Total Demand'] = preconsensus['Sum of Curr Qtr Total Demand'].astype(int)
preconsensus['Sum of Curr Qtr Total Demand'].dtype

preconsensus.insert(58,'Fill+1%',np.where( preconsensus['Sum of Curr Qtr Total Demand'] == 0,0,
                                        preconsensus['Load.1']/preconsensus['Sum of Curr Qtr Total Demand']))

mid = preconsensus['SFDC+1']
preconsensus.drop(labels=['SFDC+1'], axis=1, inplace = True)
preconsensus.insert(59, 'SFDC+1', mid)

preconsensus.insert(60,'Fill(CRM)+1%',np.where( preconsensus['Sum of Curr Qtr Total Demand'] ==0,0,
                                        (preconsensus['Load.1']+preconsensus['SFDC+1'])/preconsensus['Sum of Curr Qtr Total Demand']))

preconsensus.insert(61,'Secure Qty.1','')
preconsensus.insert(62,'Secure Count.1','')
preconsensus.insert(63,'Validate Qty.1','')
preconsensus.insert(64,'Validate Count.1','')
preconsensus.insert(65,'Qualify Qty.1','')
preconsensus.insert(66,'Qualify Count.1','')
preconsensus.insert(67,'Identify Qty.1','')
preconsensus.insert(68,'Identify Count.1','')

#arrange qty and count Q1
preconsensus['Secure Qty.1']=preconsensus['SecureQ1']
preconsensus = preconsensus.drop(['SecureQ1'], axis = 1)

preconsensus['Secure Count.1']=preconsensus['SecurecountQ1']
preconsensus = preconsensus.drop(['SecurecountQ1'], axis = 1)

preconsensus['Validate Qty.1']=preconsensus['ValidateQ1']
preconsensus = preconsensus.drop(['ValidateQ1'], axis = 1)

preconsensus['Validate Count.1']=preconsensus['ValidatecountQ1']
preconsensus = preconsensus.drop(['ValidatecountQ1'], axis = 1)

preconsensus['Qualify Qty.1']=preconsensus['QualifyQ1']
preconsensus = preconsensus.drop(['QualifyQ1'], axis = 1)

preconsensus['Qualify Count.1']=preconsensus['QualifycountQ1']
preconsensus = preconsensus.drop(['QualifycountQ1'], axis = 1)

preconsensus['Identify Qty.1']=preconsensus['IdentifyQ1']
preconsensus = preconsensus.drop(['IdentifyQ1'], axis = 1)

preconsensus['Identify Count.1']=preconsensus['IdentifycountQ1']
preconsensus = preconsensus.drop(['IdentifycountQ1'], axis = 1)
#END SFR NUMBERS FOR Q1
preconsensus.insert(69,'Wgt Funnel + Load.1',0)
preconsensus.insert(70,'Totl Funnel + Load.1',0)
preconsensus.insert(71,'Total SFDC Count.1',0)
preconsensus.insert(72,'Wgt Funnel As % of Tot Dmd.1',np.where( preconsensus['Sum of Curr Qtr Total Demand'] ==0,0,
                                        preconsensus['Wgt Funnel + Load.1']/preconsensus['Sum of Curr Qtr Total Demand']))
preconsensus.insert(73,'Tot Funnel As % of Tot Dmd.1',0)
preconsensus.insert(74,'Wgt Funnel+Load Fill.1',0)
preconsensus.insert(75,'Demand var.1',0)

#Q+2
#mid = preconsensus['LRP_2']
#preconsensus.drop(labels=['LRP_2'], axis=1, inplace = True)
#preconsensus.insert(53, 'LRP.2', mid)
preconsensus.insert(76,'LRP.2',0)
#Prev PM Outlook
preconsensus.insert(77,'PM Outlook.2',0)
preconsensus.insert(78,'Secure Qty.2','')
preconsensus.insert(79,'Secure Count.2','')
preconsensus.insert(80,'Validate Qty.2','')
preconsensus.insert(81,'Validate Count.2','')
preconsensus.insert(82,'Qualify Qty.2','')
preconsensus.insert(83,'Qualify Count.2','')
preconsensus.insert(84,'Identify Qty.2','')
preconsensus.insert(85,'Identify Count.2','')

#arrange qty and count Q2
preconsensus['Secure Qty.2']=preconsensus['SecureQ2']
preconsensus = preconsensus.drop(['SecureQ2'], axis = 1)

preconsensus['Secure Count.2']=preconsensus['SecurecountQ2']
preconsensus = preconsensus.drop(['SecurecountQ2'], axis = 1)

preconsensus['Validate Qty.2']=preconsensus['ValidateQ2']
preconsensus = preconsensus.drop(['ValidateQ2'], axis = 1)

preconsensus['Validate Count.2']=preconsensus['ValidatecountQ2']
preconsensus = preconsensus.drop(['ValidatecountQ2'], axis = 1)

preconsensus['Qualify Qty.2']=preconsensus['QualifyQ2']
preconsensus = preconsensus.drop(['QualifyQ2'], axis = 1)

preconsensus['Qualify Count.2']=preconsensus['QualifycountQ2']
preconsensus = preconsensus.drop(['QualifycountQ2'], axis = 1)

preconsensus['Identify Qty.2']=preconsensus['IdentifyQ2']
preconsensus = preconsensus.drop(['IdentifyQ2'], axis = 1)

preconsensus['Identify Count.2']=preconsensus['IdentifycountQ2']
preconsensus = preconsensus.drop(['IdentifycountQ2'], axis = 1)
#END SFR NUMBERS FOR Q1
preconsensus.insert(86,'Wgt Funnel + Load.2',0)
preconsensus.insert(87,'Total SFDC Count.2',0)

#preconsensus.drop(labels=['Fill+1%'], axis=1, inplace = True)
#preconsensus.iloc[:, 23].name
#activate formula once preconsensus['Wgt Funnel + Load.2'] is different to 0, 
#when that column itlsef has a formula
#preconsensus.insert(88,'Wgt Funnel As % of Tot Dmd.2',np.where( preconsensus.iloc[:, 23] ==0,0,preconsensus['Wgt Funnel + Load.2']/preconsensus.iloc[:, 23]))
preconsensus.insert(88,'Wgt Funnel As % of Tot Dmd.2',0)
preconsensus.insert(89,'Tot Funnel As % of Tot Dmd.2',0)
#delete columns not needed
preconsensus.drop(labels=['Sum of Fill Rate %'], axis=1, inplace = True) #from S&OP

preconsensus.sort_values(by=['Product Family','Region'],inplace=True)
#make product family as ww for formula in excel
preconsensus['Product Family'] =np.where(preconsensus['Region'] == 'WW','WW',consensus['Product Family'])

In [48]:
preconsensus.to_excel('ouput preconsensus.xlsx', index=False,startrow=2)

In [42]:
print(consensus1.shape,'file S&OP')
print(consensus.shape,'file+ww')
print(ww.shape,'size of ww')
print(preconsensus.shape,'consensus+ww+fillrate+lastweekconsensus')

(1419, 26) file S&OP
(1853, 28) file+ww
(434, 2) size of ww
(1853, 101) consensus+ww+fillrate+lastweekconsensus


In [36]:
#preconsensus.iloc[:, 23].name
consensus.head()

,Demand Planner,FAM & REG,Business,Product Family,Region,Sum of Q2'21 Booking,Sum of Q1'21 Booking,Sum of Q4'20 Booking,Sum of Q3'20 Booking,Sum of 4 Qtr Avg Bookings,...,Sum of SFSTK,Sum of Curr Qtr Total Demand,Sum of Q4'2021 Total Demand,Sum of Q1'2022 Total Demand,Sum of Q2'2022 Total Demand,Sum of Q3'2022 Total Demand,Sum of Load,Sum of Fill Rate %,Sum of Fill RatewSFDC %,Sum of Wgt Funnel
0,,SE5XXEMEA,EVM,SE5XX,EMEA,660000.0,588000,,,312000,...,72000,585600.0,468000,648000,648000,648000,585600,3.000000,3.000000,23000
1,,SftGdsEnt-AccNA,EVM,SftGdsEnt-Acc,NA,361176.0,185226,,,136600,...,90539,453919.0,610662,382166,304211,276388,348574,112.938235,112.938235,
2,,BtryEnt-AccNA,EVM,BtryEnt-Acc,NA,168200.0,101321,,,67380.2,...,34018,175534.0,361374,329205,179239,138938,124455,53.755120,53.755120,
3,,HandheldGP-AccNA,EVM,HandheldGP-Acc,NA,296436.0,318834,,,153818,...,98080,306280.0,493621,255374,254162,253588,254457,564.056239,564.056239,
4,,CblsEnt-AccNA,EVM,CblsEnt-Acc,NA,207553.0,133907,,,85365,...,67095,255577.0,270908,230292,180681,118458,245900,79.549722,79.549722,


In [37]:
fillrate.head()

,FAM & REG,Group,Sum of Roll Over Backlog,Sum of Curr Qtr Backlog,Sum of Total Curr Qtr Ship,Sum of SFDC,SFDC+1,Load.1,Sum of Q4'2021 Open Bklg,Sum of Q1'2022 Open Bklg,Sum of Q2'2022 Open Bklg,Sum of Q3'2022 Open Bklg
0,1dEngine-AccEMEA,DCS,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,1dEngine-AccLA,DCS,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,1dEngine-AccNA,DCS,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,1dEngine-AccAPAC,DCS,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
9,Antenna-AccAPAC,DCS,0.0,13.0,155.0,0.0,0.96,202.0,409.0,0.0,0.0,0.0


In [38]:
preconsensus.head()

,Demand Planner,FAM & REG,Business,Group,Technology,Product Family,Region,Sum of Q2'21 Booking,Sum of Q1'21 Booking,Sum of Q4'20 Booking,...,Sum of Wgt Funnel,Sum of Roll Over Backlog,Load,Sum of Total Curr Qtr Ship,Sum of Q4'2021 Open Bklg,Sum of Q1'2022 Open Bklg,Sum of Q2'2022 Open Bklg,Sum of Q3'2022 Open Bklg,Prev PM OUTLOOK.2,Prev PM OUTLOOK.3
829,,105SLEMEA,EVM,Printers,,SE5XX,EMEA,0.0,0,,...,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1684,,105SLWW,EVM,NaN,,WW,WW,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
830,,105SLPlusAPAC,EVM,Printers,,BtryEnt-Acc,APAC,0.0,0,,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
831,,105SLPlusEMEA,EVM,Printers,,HandheldGP-Acc,EMEA,0.0,0,,...,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
832,,105SLPlusLA,EVM,Printers,,CblsEnt-Acc,LA,0.0,0,,...,2.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [ ]:
Q.to_excel('Q32021.xlsx', index=True)

In [39]:
preconsensus.columns

Index(['Demand Planner', 'FAM & REG', 'Business', 'Group', 'Technology',
       'Product Family', 'Region', 'Sum of Q2'21 Booking',
       'Sum of Q1'21 Booking', 'Sum of Q4'20 Booking',
       ...
       'Sum of Wgt Funnel', 'Sum of Roll Over Backlog', 'Load',
       'Sum of Total Curr Qtr Ship', 'Sum of Q4'2021 Open Bklg',
       'Sum of Q1'2022 Open Bklg', 'Sum of Q2'2022 Open Bklg',
       'Sum of Q3'2022 Open Bklg', 'Prev PM OUTLOOK.2', 'Prev PM OUTLOOK.3'],
      dtype='object', length=101)

# notes
WW IN REGION AND PORFUCT FAMILY
WATCH FOR BLANKS
SFR QTY AND COUNT

In [ ]:
preconsensus.iloc[:, 23].name

In [ ]:
preconsensus.drop(labels=['Fill%'], axis=1, inplace = True)

preconsensus[['M/M Cons Chg']].head()

In [ ]:
preconsensus[preconsensus.columns[12]].head()

In [ ]:
print(consensus1.shape)
print(fillrate.shape)
print(consensus.shape)
print(ww.shape)